In [38]:
SMPL_JOINT_NAMES = {
    0:  'Pelvis',
    1:  'L_Hip',        4:  'L_Knee',            7:  'L_Ankle',           10: 'L_Foot',
    2:  'R_Hip',        5:  'R_Knee',            8:  'R_Ankle',           11: 'R_Foot',
    3:  'Spine1',       6:  'Spine2',            9:  'Spine3',            12: 'Neck',            15: 'Head',
    13: 'L_Collar',     16: 'L_Shoulder',       18: 'L_Elbow',            20: 'L_Wrist',         22: 'L_Hand',
    14: 'R_Collar',     17: 'R_Shoulder',       19: 'R_Elbow',            21: 'R_Wrist',         23: 'R_Hand',
}

def save_mesh(model, outmesh_path: str = '/data/lixinag/project/ContourCraft/test.obj'):
    with open(outmesh_path, 'w') as fp:
        for v in model.r:
            fp.write( 'v %f %f %f\n' % ( v[0], v[1], v[2]) )

        for f in model.f+1: # Faces are 1-based, not 0-based in obj files
            fp.write( 'f %d %d %d\n' %  (f[0], f[1], f[2]) )
    print('..Output mesh saved to: ', outmesh_path)
import sys
sys.path.append('/data/lixinag/project/GarmentCode/A_RESULT/smplx/SMPL_python_v.1.1.0/smpl')
from smpl_webuser.serialization import load_model

import numpy as np

In [39]:
outmesh_path = '/data/lixinag/project/ContourCraft/test.obj'
m = load_model('/data/lixinag/project/GarmentCode/A_RESULT/smplx/SMPL_FEMALE.pkl')
save_mesh(m, outmesh_path=outmesh_path)
import trimesh
smpl_ori_mesh = trimesh.load(outmesh_path)
smpl_ori_mesh.show()

..Output mesh saved to:  /data/lixinag/project/ContourCraft/test.obj


In [40]:
import trimesh
Apose = trimesh.load("/data/lixinag/project/ContourCraft/ccraft_data/examples/femaleA40pose/smpl_body.obj", process=False)
Apose.show()

In [41]:
AMASS_smpl_sample_path = "/data/lixinag/project/ContourCraft/AMASS/smpl/CMU/01/01_01_poses.npz"
motion_data = dict(np.load(AMASS_smpl_sample_path, allow_pickle=True))

In [43]:
motion_data["poses"].shape
m.pose[:] = motion_data["poses"][100][:72]
m.pose[:3] = 0
save_mesh(m, outmesh_path=outmesh_path)
import trimesh
mesh = trimesh.load(outmesh_path)
mesh.show()

..Output mesh saved to:  /data/lixinag/project/ContourCraft/test.obj


In [47]:
def get_verts_from_motion_data(model, motion_path, index):
    motion_poses_data = dict(np.load(motion_path, allow_pickle=True))["poses"]
    model.pose[:] = motion_poses_data[index][:72]
    model.pose[:3] = 0
    return model.r

In [48]:
def generate_body_verts(body_src_vert, body_tgt_vert, n_frames):
    weights = np.linspace(0, 1, n_frames)
    body_verts = np.zeros((n_frames, body_src_vert.shape[0], 3))
    for i in range(n_frames):
        body_verts[i] = body_src_vert * (1 - weights[i]) + body_tgt_vert * weights[i]
    return body_verts

In [49]:
n_frames = 48
t_index = 100

s = np.array(Apose.vertices)
t = get_verts_from_motion_data(m, AMASS_smpl_sample_path, t_index)

body_pkl = {}
body_pkl['verts'] = generate_body_verts(s, t, n_frames)
body_pkl['faces'] = np.array(mesh.faces)
print(f"{body_pkl['verts'].shape = }, {body_pkl['faces'].shape = }")
import pickle
pickle.dump(body_pkl, open('/data/lixinag/project/ContourCraft/ccraft_data/examples/femaleA40pose/smpl_body1.pkl', 'wb'))

body_pkl['verts'].shape = (48, 6890, 3), body_pkl['faces'].shape = (13776, 3)
